# Task 2: a more challenging percolation problem

In this notebook, we are going to simulate a percolating substance that is able to propagate horizontally, as well as downwards.

As a physical motivation, we might think of the percolating substance as water, and the simulation as representing water percolating through soil under gravity.
This is actually a very relevant issue today; for example, understanding the way in which water percolates through different substances is crucial for accurate flood risk assessments, and flood mitigation strategies often aim to slow down the speed of percolation, without stopping it (which would lead to surface runoff and make the situation worse).

The aim of this notebook will be to determine the value of $q$ - i.e. `frozen_prob` - at which percolation switches from becoming more than 50\% probable, to less than 50\% probable.

<div class="alert alert-block alert-success">
    <b>Definition:</b> We will define the <b>percolation threshold</b> $q^*$ as the value of $q$ which corresponds to a percolation probability of $p = \frac{1}{2}$.
</div>

## Running this variation of the model

### Importing the code

Of course, the first thing we need to do is import the relevant bits of code.

In [ ]:
from p1b_percolation.lattice import SquareLattice
from p1b_percolation.model import PercolationModel
from p1b_percolation.scripts.parameter_scan import parameter_scan

### Setting up the model

We're going to provide a second argument to `SquareLattice`, called `n_links`.
In the previous notebook, `n_links` took its default value of 1, but we now want to change it to 3.

Run the following cell to see the effect of changing `n_links`.

In [ ]:
help(SquareLattice.n_links)

The way we will create a $50\times 50$ lattice is
```python
lattice = SquareLattice(50, 50, n_links=3)
```

You might wonder why we don't just write
```python
lattice = SquareLattice(50, 50, 3)
```
Python allows both, but it is usually preferable to explicitly state the name of the argument which you are providing a value for.
This is particularly helpful when a function has many arguments, and you don't want to rely on remembering the order in which they are supposed to appear.

We could have been even more explicit and written
```python
lattice = SquareLattice(n_rows=50, n_cols=50, n_links=3)
```
but it's easy to remember that the first two arguments specify the size of the lattice.

In [ ]:
lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice, frozen_prob=0.38)

model.animate(100)

## The percolation transition

To estimate the percolation probability $p$, we will use a method similar to one you saw in the previous notebook.
We will run a number, `repeats`, of separate simulations and record the fraction of these that percolated,

$$
f = \frac{1}{N} \sum_{i=1}^N b_i \tag{Estimator}
$$

where $b_i$ are equal to either 0 (for simulations which don't percolate) or 1 (for those that do), and $N$ is the number of simulations (`repeats`).


### Errors

Just as before, $N \times f$ still follows a binomial distribution, since it is the sum of binary variables $b_i$ (these are also called 'Bernoulli trials').
Thus, the formula for the standard error on $f$ remains the same:

$$
\sigma_f = \sqrt{\frac{p(1 - p)}{N}} \tag{Standard error}
$$

Previously, we calculated $p$ by hand using basic probability theory.
Unfortunately, in this (and almost every interesting) situation, we can't just calculate the answer by hand, otherwise we wouldn't bother spending all this effort estimating it using simulations!

The standard approach in situations like this is to replace $p$ with its estimate, i.e. $f$.

The cell below also calculates an estimate of the standard error using $f$ in place of $p$.

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
    Run the cell for `frozen_prob` equal to 0.34, 0.4, and 0.45, and for 10, 100 and 1000 repeats, and record the results in your lab book.
</div>

In [ ]:
frozen_prob = 0.34  # change this value!
repeats = 10  # and this one!

lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice, frozen_prob)

model.estimate_percolation_prob(repeats, print_error=True)

This cell is left intentionally blank.

You should see that the estimate of the standard error can be zero, when $f$ is either 0 or 1.
Of course, it is not correct to conclude that the error is zero when $f=0$ or $f=1$; we cannot say with 100\% certainty whether the next simulation will percolate or not.

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
    Discuss this observation in your lab book. What's going wrong? What is the effect of increasing $N$? Why?
</div>

**Hint:** This example illustrates why it is so important to distinguish between the estimator, $f$ (which is only as good as your experiment!), and the thing being estimated, $p$.

<div class="alert alert-block alert-danger">
<b>Note for CO+TAs</b> Not sure whether to go on to say we used a different error for these points, or get them to try to avoid these points in their parameter scans. I would personally opt for the former.
</div>

## Finding the percolation threshold using a 'fit'

<div class="alert alert-block alert-success">
    <b>Definition:</b> A <b>fit</b> is when you take a function and carefully tune the parameters so that the resulting line matches your data as well as possible. You are probably very familiar with the "line of best fit", which corresponds to the function $f(x) = m x + c$, whose parameters are $m$ (the gradient) and $c$ (the y-intercept).
</div>

Interested students can take 5 minutes at this point to have a read through the section [Further reading: Fitting functions to data](#Further-reading:-Fitting-functions-to-data) before returning to this point.

We will attempt to fit a **Logistic** function to our simulation data

$$
S(q) = \frac{1}{1 + e^{\lambda (q - q_0)}}
$$

$S(q)$ is a very simple function that smoothly transitions between the asymptotes

$$
\lim_{q\to-\infty} S(q) = 1 \qquad\qquad
\lim_{q\to\infty} S(q) = 0
$$

and has two *parameters* (just like the line of best fit):
* The mid-point $q_0$, such that $S(q_0) = \frac{1}{2}$. We will use this to *estimate* the percolation threshold, $q^*$.
* The 'steepness' $\lambda$, which controls how quickly the transition from approximately 1 to approximately 0 occurs.


Run the cell below.

In [ ]:
lattice = SquareLattice(50, 50, n_links=3)
model = PercolationModel(lattice)

parameter_scan(model, start=0.35, stop=0.45, num=50, repeats=25)

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
Comment on the quality of the logistic curve fit.
Do the residuals show any obvious structure, or are they uniformly distributed above and below the fit curve?
</div>

The Logistic function is the simplest function that has 'essentially the correct shape', and since we don't have any other clues (e.g. from an underlying theory), it is a good function to start with in this case.
To emphasise the point again, it is important to realise that we do not have any fundamental reason to think $S(q)$ is going to fit the data perfectly, but we have to start somewhere!

## Extrapolating to infinite lattice size

There is one glaring issue with all the analysis we have done so far; all of our results depend on the size of the lattice!

So-called **finite-volume effects** are an inevitability with simulations, since we cannot simulate an infinite system.
However, it is possible to *extrapolate* results to the infinite limit, by looking at how the thing you're measuring varies as the size of simulations increases.
The answers you get after extrapolating are more likely to agree with the physical world.

We will attempt to extract some infinite-lattice results for the percolation threshold, $q^*(\infty)$.

You will need to run the cell below for a range of different values of `n_rows`, the number of rows (the number of columns will automatically be set equal to this).
Let's label the number of rows as $L$.

Record your results in a table, making sure to note down the associated errors on $q_0$ **with an appropriate precision**.
Also record your values for `start` and `stop` in the table; these will need to be carefully selected for each lattice size so that almost all of the data points take values between 0 and 1, but are not 0 or 1.
**Leave three blank columns in your table.**

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
Record your results in a table.
</div>

For example, your table could look like this

| Lattice size ($L$, `n_rows`) | Min $q$ (`start`) | Max $q$ (`stop`) | Mid point ($q_0$) | Error ($\sigma_{q_0}$) | - | - | - |
| --- | --- | --- | --- | --- | --- | --- | --- |
| ... | ... | ... | ... | ... | ... | ... | ... |

Keep a note of what you used for`num` and `repeats`, but since there is no need to vary them they do not require a column in the table.

Aim to gather results for 5-10 different lattice sizes ranging between `n_rows = 20` and `n_rows = 200`.

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    The larger simulations will take a while to complete. There is no need to generate more than 25 data points for each lattice size, and 25 repeats should also suffice. The largest lattice size may take up to 5 minutes, so grab a cup of coffee while you wait!
</div>

In [ ]:
n_rows = 20  # change this!
n_cols = n_rows

lattice = SquareLattice(n_rows, n_cols, n_links=3)
model = PercolationModel(lattice)

parameter_scan(model, start=0.3, stop=0.5, num=35, repeats=25)

You may be able to tell, by just studying the numbers in your table, that the relationship between lattice size (`n_rows`) and the mid-point of the transition is not linear.

We really can't say *a priori* what the relationship is, but it's often a good next step to take logarithms to check if the relationship is a power-law.


<div class="alert alert-block alert-info">
<b>Lab book:</b> 
In the remaining three columns of your table, calculate the logarithm of the lattice size, the logarithm of $q_0$, and the error on $\log q_0$.
</div>

This will require you to *propagate* the error on $q_0$ through the logarithm.
Refer to the lab manual if you need to double check how to do this.

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
Plot a graph of $\log L$ v.s. $\log q_0(L)$ and draw a line of best fit.
Calculate the gradient of your line, with an estimate of its error.
</div>

Now, let's say we take the lattice size to tend towards infinity (i.e. we move right-wards along the horizontal axis).

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
What does your graph predict as the position of the percolation transition for an infinite lattice, $q_0(\infty)$?
Can you explain this result?
</div>

This cell is left intentionally blank.

<div class="alert alert-block alert-info">
<b>Lab book:</b> 
Discuss the limitations of this model to accurately describe water percolating through soil under gravity.
How might you extent the model to make it more realistic?
</div>

# Further reading: Fitting functions to data

<div class="alert alert-block alert-warning">
<b>Warning!</b> 
    Instead of talking of fitting <b>functions</b> to data, it is more common to speak of fitting a <b>model</b> to some observations.
    In this situation, a model is just a construct which, through the correct choice of its parameters, purports to be able to predict the observations.
</div>

Throughout your time at school or college you may have spent a good deal of time drawing straight lines through data on paper, or perhaps even using something like Excel.

*Linear fits* are exceptionally useful, and as such, if it is possible to manipulate your data into a form where it is fit well by a straight line, then this is the approach you should take.
As an example, we frequently take the logarithm of our variables if they obey a power law $y = A x^\alpha$, since the variables $\log y$ and $\log x$ are related by the *linear* equation $\log y = \alpha \log x + \log A$, which is much easier to work with.

However, it is not always possible to find a form in which our data is fit well by a straight line.
We might imagine that there are more suitable functions that fit our data better.
For example, if we were measuring the voltage of mains AC as a function of time, this is likely to be fit well by a sinusoid $V(t) = A \sin(100\pi t + \varphi)$, where we can tune the amplitude $A$ and the phase shift $\varphi$ (the frequency we know is 50Hz) to get the best fit.
In contrast, there is no way we can tune the gradient $m$ and the y-intercept $c$ to make the straight line $V(t) = m t + c$ fit the data well!

But drawing a general function by hand is very difficult!
How do we *actually do the fit*?
This is a very complicated question, but all you need to understand at this point is that there are *algorithms* that *automatically* tune the *free parameters* of your function so that it fits the data as well as possible.

The most important of these is called the **method of least squares**.
The way this algorithm works is by repeatedly 'nudging' the free parameters so that each nudge results in a reduction in a quantity which encapsulates how well the function fits the data.
For the curious, this quantity is result of adding up the *squared differences* between each data point and the function that you are trying to fit.
This is the square of the residuals that we've been plotting.

However, with such power comes, as always, great responsibility.
The famous mathematician **John von Neumann** once said

> "With four free parameters I can fit an elephant, and with five I can make him wiggle his trunk!"

What he means is, you could come up with an extremely complicated function, with many many free parameters, and the least squares algorithm could twiddle these parameters so that the function passed through all of the error bars on your data points (which could be in the shape of an elephant!)
This *does not mean that your function is a good representation of the underlying laws that generated the data*.

If you're struggling to follow this, don't worry!
Just bear it in mind as you continue your degree:
**If two functions fit your data equally well, and one has fewer free parameters than the other, then you should choose the one with fewer parameters.**
This is a version of **Occam's razor**, which basically says that "the simplest theory that accurately predicts the phenomena is usually the correct one".


